<a href="https://colab.research.google.com/github/ArneHei/Backend_Mobility/blob/main/Backend_Mobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install ipywidgets SQLAlchemy --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.7 MB/s eta 0:00:00


In [ ]:
#import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, Markdown
import pandas as pd
from google.colab import userdata
from google.colab import files
from google.colab import drive
import datetime
import numpy as np
import uuid
import sys
import os
import nbformat
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, Float, Boolean, Date, Time
from sqlalchemy.orm import sessionmaker, declarative_base

drive.mount('/content/drive')

# Run other colabs
#%run -i "/content/drive/MyDrive/Driver app/Data generator.ipynb"
%run -i "/content/drive/MyDrive/Driver app/Classes.ipynb"
%run -i "/content/drive/MyDrive/Driver app/Functions.ipynb"


#GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
#genai.configure(api_key=GOOGLE_API_KEY)
#gemini_model = genai.GenerativeModel('models/gemini-2.5-pro')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#"""
# Unmount Google Drive (if already mounted)
try:
    drive.flush_and_unmount()
    print("Google Drive unmounted.")
except ValueError:
    print("Google Drive was not mounted or already unmounted.")

# Remount Google Drive
drive.mount('/content/drive')
print("Google Drive remounted. Please re-run any cells that load data from Drive to ensure they pick up the latest versions.")
#"""

Google Drive unmounted.
Mounted at /content/drive
Google Drive remounted. Please re-run any cells that load data from Drive to ensure they pick up the latest versions.


# Create classes to load in Data

In [ ]:
# Create the engine to connect to the existing database file on Google Drive
Base = declarative_base()


class ShipmentDB(Base):
    __tablename__ = 'shipments'

    Shipment_ID = Column(String, primary_key=True)
    Transport = Column(String)
    Department = Column(String)
    Pickup_time = Column(String)
    Pickup_date = Column(Date)
    Delivery_time = Column(String)
    Delivery_date = Column(Date)
    Collection_Name = Column(String)
    Collection_City = Column(String)
    Collection_Address = Column(String)
    Collection_Postal_Code = Column(String)
    Collection_Country = Column(String)
    Delivery_Name = Column(String)
    Delivery_City = Column(String)
    Delivery_Address = Column(String)
    Delivery_Postal_Code = Column(String)
    Delivery_Country = Column(String)
    Weight = Column(Float)
    Volume = Column(Float)
    Ldm = Column(Float)
    Content = Column(String)
    Units = Column(Integer)
    Unit_type = Column(String)
    Hazardous = Column(Boolean)
    Cost = Column(Float)
    Finance_Department = Column(String)
    Incoterm = Column(String)
    Customer = Column(String)
    Loading_Instructions = Column(String)
    Customer_Reference = Column(String)
    Additional_Information = Column(String)
    Services = Column(String)

    def __repr__(self):
        return f"<ShipmentDB(Shipment_ID='{self.Shipment_ID}', Department='{self.Department}')>"

class TruckDB(Base):
    __tablename__ = 'trucks'
    License_plate = Column(String, primary_key=True, unique=True, nullable=False)
    Driver = Column(String)
    Trailer = Column(String)
    Phone = Column(String)
    Haulier = Column(String)
    Time = Column(String)
    Date = Column(String)
    Location = Column(String)
    Transport = Column(String)
    Department = Column(String)
    Pay_type = Column(String)
    Pay_amount = Column(Float)

    def __repr__(self):
        return f"<TruckDB(License_plate='{self.License_plate}', Haulier='{self.Haulier}')>"

class TrailerDB(Base):
    __tablename__ = 'trailers'
    __table_args__ = {'extend_existing': True} # Add this line
    License_plate = Column(String, primary_key=True, unique=True, nullable=False)
    Type = Column(String)
    Sub_type = Column(String)
    Department = Column(String)
    Open_pool = Column(Boolean)
    Transport = Column(String, nullable=True)
    Start_date = Column(Date, nullable=True)
    End_date = Column(Date, nullable=True)

    def __repr__(self):
        return f"<TrailerDB(License_plate='{self.License_plate}', Department='{self.Department}', Type='{self.Type}')>"



# Load in Shipments

In [ ]:
#Define path to Shipments
db_path = "/content/drive/MyDrive/Driver app/FTL_LTL_ship.db"
engine = create_engine(f'sqlite:///{db_path}')

# Create all tables in the engine.
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Query all shipments from the database
all_shipments_from_db = session.query(ShipmentDB).all()

# Populate the Shipment class registry
for s in all_shipments_from_db:
    Shipment(
        Shipment_ID=s.Shipment_ID,
        Transport=s.Transport,
        Department=s.Department,
        Pickup_time=s.Pickup_time,
        Pickup_date=s.Pickup_date,
        Delivery_time=s.Delivery_time,
        Delivery_date=s.Delivery_date,
        Collection_Name=s.Collection_Name,
        Collection_City=s.Collection_City,
        Collection_Address=s.Collection_Address,
        Collection_Postal_Code=s.Collection_Postal_Code,
        Collection_Country=s.Collection_Country,
        Delivery_Name=s.Delivery_Name,
        Delivery_City=s.Delivery_City,
        Delivery_Address=s.Delivery_Address,
        Delivery_Postal_Code=s.Delivery_Postal_Code,
        Delivery_Country=s.Delivery_Country,
        Weight=s.Weight,
        Volume=s.Volume,
        Ldm=s.Ldm,
        Content=s.Content,
        Units=s.Units,
        Unit_type=s.Unit_type,
        Hazardous=s.Hazardous,
        Cost=s.Cost,
        Finance_Department=s.Finance_Department,
        Incoterm=s.Incoterm,
        Customer=s.Customer,
        Loading_Instructions=s.Loading_Instructions,
        Customer_Reference=s.Customer_Reference,
        Additional_Information=s.Additional_Information
    )

# Convert to a list of dictionaries for easier display or DataFrame creation
data_for_df = []
for s in all_shipments_from_db:
    data_for_df.append({
        'Shipment_ID': s.Shipment_ID, # Include Shipment_ID
        'Transport': s.Transport,
        'Department': s.Department,
        'Pickup_time': s.Pickup_time,
        'Pickup_date': s.Pickup_date,
        'Delivery_time': s.Delivery_time,
        'Delivery_date': s.Delivery_date,
        'Collection_Name': s.Collection_Name,
        'Collection_City': s.Collection_City,
        'Collection_Address': s.Collection_Address,
        'Collection_Postal_Code': s.Collection_Postal_Code,
        'Collection_Country':s.Collection_Country,
        'Delivery_Name': s.Delivery_Name,
        'Delivery_City': s.Delivery_City,
        'Delivery_Address': s.Delivery_Address,
        'Delivery_Postal_Code': s.Delivery_Postal_Code,
        'Delivery_Country':s.Delivery_Country,
        'Weight': s.Weight,
        'Volume': s.Volume,
        'Ldm': s.Ldm,
        'Content': s.Content,
        'Units': s.Units,
        'Unit_type': s.Unit_type,
        'Hazardous': s.Hazardous,
        'Cost': s.Cost,
        'Finance_Department':s.Finance_Department,
        'Incoterm':s.Incoterm,
        'Customer':s.Customer,
        'Loading_Instructions':s.Loading_Instructions,
        'Customer_Reference':s.Customer_Reference,
        'Additional_Information':s.Additional_Information
    })

df_from_db = pd.DataFrame(data_for_df)

print(f"Successfully loaded {len(df_from_db)} shipments from the database.")
display(df_from_db.head())

# Close the session
session.close()

Successfully loaded 5289 shipments from the database.


,Shipment_ID,Transport,Department,Pickup_time,Pickup_date,Delivery_time,Delivery_date,Collection_Name,Collection_City,Collection_Address,...,Units,Unit_type,Hazardous,Cost,Finance_Department,Incoterm,Customer,Loading_Instructions,Customer_Reference,Additional_Information
0,BAHN4-00ZKI,,TEST1,00:01-23:59,2026-01-26,00:01-23:59,2026-01-26,BAHNE C/O SOLUTIONS,HORSENS,MOSSVEJ 27 PORT 718-722,...,10,PLL,False,5100.0,,,,,,
1,BOLIA-D0PTJ,,TEST1,08:00-12:00,2026-02-03,00:01-23:59,2026-02-05,AAGE ØSTERGAARD MØBELPRODUKTION,ROSLEV,"HESTHØJVEJ 16, PORT 9 DK-7870 ROSLEV",...,69,CTN,False,4000.0,,,,,,
2,BOLIA-D1KEG,,TEST1,00:01-23:59,2026-02-05,00:01-23:59,2026-02-05,MASSIMO COPENHAGEN APS.,BIRKERØD,BLOKKEN 79 +45 48 42 25 73,...,600,CTN,False,2100.0,,,,,,
3,BOLIA-D1P6P,,TEST1,11:00-11:00,2026-01-26,10:00-19:00,2026-01-26,AAGE ØSTERGAARD MØBELPRODUKTION,ROSLEV,"HESTHØJVEJ 16, PORT 9 DK-7870 ROSLEV",...,95,CTN,False,5400.0,,,,,,
4,BOLIA-D1AK9,,TEST1,00:01-23:59,2026-01-26,00:01-23:59,2026-01-27,"SANYEI SCANDINAVIA A/S, LAGER",TÅRS,VESTER BORUPVEJ 20 - PORT 6 DK-9830 TÅRS,...,106,CTN,False,5000.0,,,,,,


In [ ]:
print("\nVerifying database content...")
engine_verify = create_engine(f'sqlite:///{db_path}')
Session_verify = sessionmaker(bind=engine_verify)
session_verify = Session_verify()
try:
    # Query the first 5 records
    first_5_shipments = session_verify.query(ShipmentDB).limit(5).all()

    if first_5_shipments:
        print("First 5 records from the database:")
        for shipment in first_5_shipments:
            print(f"  Shipment_ID: {shipment.Shipment_ID}, Pickup_date: {shipment.Pickup_date}, Delivery_date: {shipment.Delivery_date}")
    else:
        print("No records found in the database.")
except Exception as e:
    print(f"Error querying database: {e}")
finally:
    if session_verify:
        session_verify.close()
    print("Verification database session closed.")


Verifying database content...
First 5 records from the database:
  Shipment_ID: BAHN4-00ZKI, Pickup_date: 2026-01-26, Delivery_date: 2026-01-26
  Shipment_ID: BOLIA-D0PTJ, Pickup_date: 2026-02-03, Delivery_date: 2026-02-05
  Shipment_ID: BOLIA-D1KEG, Pickup_date: 2026-02-05, Delivery_date: 2026-02-05
  Shipment_ID: BOLIA-D1P6P, Pickup_date: 2026-01-26, Delivery_date: 2026-01-26
  Shipment_ID: BOLIA-D1AK9, Pickup_date: 2026-01-26, Delivery_date: 2026-01-27
Verification database session closed.


# Load in Trucks

In [ ]:
# 1. Define the path to the 'Trucks.db' database
truck_db_path = '/content/drive/MyDrive/Driver app/Trucks.db'

# 2. Create a SQLAlchemy engine to connect to this database
truck_engine = create_engine(f'sqlite:///{truck_db_path}')

# 3. Create a session factory and a session instance
TruckSession = sessionmaker(bind=truck_engine)
truck_session = TruckSession()

# 4. Query all records from the 'trucks' table using the TruckDB model
all_trucks_from_db = truck_session.query(TruckDB).all()

# 5. Convert the queried truck objects into a list of dictionaries
data_for_truck_df = []
for t in all_trucks_from_db:
    data_for_truck_df.append({
        'License_plate': t.License_plate,
        'Driver': t.Driver,
        'Trailer': t.Trailer,
        'Phone': t.Phone,
        'Haulier': t.Haulier,
        'Time': t.Time,
        'Date': t.Date,
        'Location': t.Location,
        'Transport': t.Transport,
        'Department': t.Department,
        'Pay_type': t.Pay_type,
        'Pay_amount': t.Pay_amount
    })

# 6. Create a pandas DataFrame from this list of dictionaries
df_trucks = pd.DataFrame(data_for_truck_df)

# 7. Print a confirmation message
print(f"Successfully loaded {len(df_trucks)} trucks from the database.")

# 8. Display the head of the DataFrame
display(df_trucks.tail())

# 9. Close the database session
truck_session.close()

# Populate the Truck class registry from df_trucks
for index, row in df_trucks.iterrows():
    Truck(
        License_plate=row['License_plate'],
        Driver=row['Driver'],
        Trailer=row['Trailer'],
        Phone=row['Phone'],
        Haulier=row['Haulier'],
        Time=row['Time'],
        Date=row['Date'],
        Location=row['Location'],
        Transport=row['Transport'],
        Department=row['Department'],
        Pay_type=row['Pay_type'],
        Pay_amount=row['Pay_amount']
    )

print(f"Successfully loaded {len(Truck.registry)} Truck objects into the registry.")

Successfully loaded 60 trucks from the database.


,License_plate,Driver,Trailer,Phone,Haulier,Time,Date,Location,Transport,Department,Pay_type,Pay_amount
55,HN53972,Anne Nørgaard,BO9848,+45 34321293,AK NEMT TRANSPORT APS,,,DK-8400,,TEST1,,0.0
56,PA63845,Noah Sørensen,KT6970,+45 57084483,LL-Transport ApS,,,DK-9600,,TEST1,,0.0
57,EO92291,Anders Dam,BN2545,+45 88293971,Dilligencen ApS,,,DK-6100,,TEST1,,0.0
58,CE85560,Maria Poulsen,LF0217,+45 00627357,Køl og Frys Transport APS,,,DK-3700,,TEST1,,0.0
59,DB10467,Tina Jensen,KI1461,+45 38525545,AK NEMT TRANSPORT APS,,,DK-2500,,TEST1,,0.0


Successfully loaded 60 Truck objects into the registry.


# Load in Trailers

In [ ]:
print('\n--- Reloading Trailer Objects from Trailers.db ---')

# 1. Define the path to the 'Trailers.db' database (already defined, but for clarity)
trailer_db_path = '/content/drive/MyDrive/Driver app/Trailers.db'

# 2. Create a SQLAlchemy engine to connect to this database
trailer_engine = create_engine(f'sqlite:///{trailer_db_path}')

# 3. Create a session factory and a session instance
TrailerDBSession = sessionmaker(bind=trailer_engine)
trailer_session = TrailerDBSession()

# 4. Query all records from the 'trailers' table using the TrailerDB model
all_trailers_from_db = trailer_session.query(TrailerDB).all()

# 5. Populate the Trailer class registry from the queried TrailerDB objects
for t in all_trailers_from_db:
    Trailer(
        License_plate=t.License_plate,
        Type=t.Type,
        Sub_type=t.Sub_type,
        Department=t.Department,
        Open_pool=t.Open_pool,
        # Include new fields when recreating Trailer objects
        Transport=t.Transport,
        Start_date=t.Start_date,
        End_date=t.End_date
    )

# 6. Close the database session
trailer_session.close()

# 7. Print a confirmation message and verify a sample
print(f"Successfully reloaded {len(Trailer.registry)} Trailer objects into the registry.")
print('\nFirst 5 reloaded Trailer objects in registry:')
count = 0
for lp, trailer_obj in Trailer.registry.items():
    if count >= 5:
        break
    print(trailer_obj)
    count += 1


--- Reloading Trailer Objects from Trailers.db ---
Successfully reloaded 50 Trailer objects into the registry.

First 5 reloaded Trailer objects in registry:
<Trailer(License_plate='WC8195', Department='TEST1', Type='Curtain', Sub type = '')>
<Trailer(License_plate='RB6987', Department='TEST1', Type='Box', Sub type = '')>
<Trailer(License_plate='LF4648', Department='TEST1', Type='Curtain', Sub type = 'Raisable')>
<Trailer(License_plate='PP1053', Department='TEST1', Type='Box', Sub type = 'Raisable')>
<Trailer(License_plate='LF0217', Department='TEST1', Type='Reefer', Sub type = 'Raisable')>


In [ ]:
List_of_Shipments = df_from_db[0:2]
List_of_Shipments2 = df_from_db[15:17]
display(List_of_Shipments)
display(List_of_Shipments2)

,Shipment_ID,Transport,Department,Pickup_time,Pickup_date,Delivery_time,Delivery_date,Collection_Name,Collection_City,Collection_Address,...,Delivery_Address,Delivery_Postal_Code,Weight,Volume,Ldm,Content,Units,Unit_type,Hazardous,Cost
0,BOLIA-D4732,None,TEST1,08:00-18:00,2026-01-16,08:00-11:00,2026-01-19,BOLIA TERMINAL - DSV TERMINAL,HORSENS,"MOSSVEJ 27 , HAL F2 HENVENDELSE V/ PORT 620",...,C/O RTC TRANSPORT/+45 70270040 ABILDAGER 8 - 14,2605.0,3915.0,42.895,10.7,Furniture,210,CLL,False,8400.0
1,BOLIA-D4497,None,TEST1,08:00-18:00,2026-01-16,08:00-11:00,2026-01-19,BOLIA TERMINAL - DSV TERMINAL,HORSENS,"MOSSVEJ 27 , HAL F2 HENVENDELSE V/ PORT 620",...,C/O RTC TRANSPORT/+45 70270040 ABILDAGER 8 - 14,2605.0,1723.0,33.816,8.5,FURNITURE,85,CLL,False,7300.0


,Shipment_ID,Transport,Department,Pickup_time,Pickup_date,Delivery_time,Delivery_date,Collection_Name,Collection_City,Collection_Address,...,Delivery_Address,Delivery_Postal_Code,Weight,Volume,Ldm,Content,Units,Unit_type,Hazardous,Cost
15,KEY04-AN050,None,TEST1,00:01-23:59,2026-01-05,00:01-23:59,2026-01-06,BYGMA HVALSÖ,HVALSØ,BENTSENSVEJ 12 DK-4330 HVALSØ,...,Logistikparken 11 Port 315-316,8220.0,1665.0,9.800,3.0,INDV.DÖRE OG KARME C,13,CLL,False,1400.0
16,KEY04-AN800,None,TEST1,00:01-23:59,2026-01-12,07:00-07:00,2026-01-13,FAERCH A/S,HOLSTEBRO,RASMUS FÆRCHS VEJ 1 DK-7500 HOLSTEBRO,...,GREVE MAIN 30 DK-2670 GREVE,2670.0,3699.0,36.906,7.2,UNSPECIFIED GOODS,35,EUR,False,5800.0


In [ ]:
Test = Transport_create(List_of_Shipments)
Test2 = Transport_create(List_of_Shipments2)

print(Test)
print(Test2)

Department set to TEST1 and Created Transport object with ID: TOUR01-0001
Department set to TEST1 and Created Transport object with ID: TOUR01-0002
<Transport(ID='TOUR01-0001', Department='TEST1', Shipments=['BOLIA-D4732', 'BOLIA-D4497'], Weight=5638.0, Volume=76.71100000000001, Ldm=19.2, status=Planning, vehicle=, Haulier=, Driver=, Trailer=, Cost=15700.0)>
<Transport(ID='TOUR01-0002', Department='TEST1', Shipments=['KEY04-AN050', 'KEY04-AN800'], Weight=5364.0, Volume=46.706, Ldm=10.2, status=Planning, vehicle=, Haulier=, Driver=, Trailer=, Cost=7200.0)>


In [ ]:
print('---- Testing Transport_add Function ----')

# 1. Create a new Transport object with some unassigned shipments
#    Get some shipments that are currently unassigned (Transport is 'None')
unassigned_shipments_df = df_from_db[df_from_db['Transport'].astype(str) == 'None'].copy()

if unassigned_shipments_df.empty:
    print("No unassigned shipments found in df_from_db to create initial transport or add to it.")
else:
    # Take the first 2 unassigned shipments for the initial transport
    initial_shipments = unassigned_shipments_df.iloc[2:4]
    if not initial_shipments.empty:
        try:
            new_transport = Transport_create(initial_shipments)
            print(f"Initial Transport created: {new_transport}")

            # Take the next 2 unassigned shipments to add
            shipments_to_add = unassigned_shipments_df.iloc[22:24]
            if not shipments_to_add.empty:
                print(f"Shipments to add: {shipments_to_add['Shipment_ID'].tolist()}")
                # 2. Call Transport_add to add more shipments to the new_transport
                updated_transport = Transport_add(new_transport.Transport_ID, shipments_to_add)

                # 3. Verify the updated Transport object
                if updated_transport:
                    print('\n---- Verification of Updated Transport ----')
                    print(f"Transport ID: {updated_transport.Transport_ID}")
                    print(f"Shipment IDs: {updated_transport.Shipments}") # Changed to .Shipments
                    print(f"Total Weight: {updated_transport.Weight}")
                    print(f"Total Volume: {updated_transport.Volume}")
                    print(f"Total Ldm: {updated_transport.Ldm}")
                    print(f"Earliest Pickup Date: {updated_transport.Pickup_date}")
                    print(f"Latest Delivery Date: {updated_transport.Delivery_date}")
                    print(f"Cost: {updated_transport.Cost}")

                    # Also verify one of the added shipment's Transport attribute
                    if shipments_to_add.iloc[0]['Shipment_ID']:
                        verified_shipment = Shipment.get_by_id(shipments_to_add.iloc[0]['Shipment_ID'])
                        if verified_shipment:
                            print(f"Verified Shipment '{verified_shipment.Shipment_ID}' Transport: {verified_shipment.Transport}")
                        else:
                            print(f"Error: Added shipment '{shipments_to_add.iloc[0]['Shipment_ID']}' not found in registry.")
                    else:
                        print("Error: shipments_to_add is empty or missing Shipment_ID.")

                else:
                    print("Error: Transport_add returned None.")
            else:
                print("Not enough unassigned shipments to test adding.")
        except ValueError as e:
            print(f"Error creating transport: {e}")
    else:
        print("Not enough unassigned shipments to create initial transport.")

---- Testing Transport_add Function ----
Department set to TEST1 and Created Transport object with ID: TOUR01-0003
Initial Transport created: <Transport(ID='TOUR01-0003', Department='TEST1', Shipments=['CTHO1-46684', 'CTHO1-46513'], Weight=2702.0, Volume=25.520000000000003, Ldm=6.0, status=Planning, vehicle=, Haulier=, Driver=, Trailer=, Cost=4000.0)>
Shipments to add: ['NAESJ-DR307', 'NAESJ-DR112']
Successfully added shipments ['NAESJ-DR307', 'NAESJ-DR112'] to Transport 'TOUR01-0003'.
Updated Transport details: Weight=9868.0, Volume=25.520000000000003, Ldm=12.8, Cost=11500.0, Stops=[<Stop(ID='CTHO1-46684_P', Type='P', City='ODENSE S', Date='2026-01-19')>, <Stop(ID='CTHO1-46513_P', Type='P', City='ODENSE S', Date='2026-01-19')>, <Stop(ID='CTHO1-46684_D', Type='D', City='VINDERUP', Date='2026-01-20')>, <Stop(ID='CTHO1-46513_D', Type='D', City='ROSKILDE', Date='2026-01-20')>, <Stop(ID='NAESJ-DR307_P', Type='P', City='SKJERN', Date='2026-01-16')>, <Stop(ID='NAESJ-DR112_P', Type='P', City=

In [ ]:
print("---- All Current Transport Objects ----")
if Transport.registry:
    for transport_id, transport_obj in Transport.registry.items():
        print(transport_obj)
else:
    print("No Transport objects currently registered.")

---- All Current Transport Objects ----
<Transport(ID='TOUR01-0001', Department='TEST1', Shipments=['BOLIA-D4732', 'BOLIA-D4497'], Weight=5638.0, Volume=76.71100000000001, Ldm=19.2, status=Planning, vehicle=, Haulier=, Driver=, Trailer=, Cost=15700.0)>
<Transport(ID='TOUR01-0002', Department='TEST1', Shipments=['KEY04-AN050', 'KEY04-AN800'], Weight=5364.0, Volume=46.706, Ldm=10.2, status=Planning, vehicle=, Haulier=, Driver=, Trailer=, Cost=7200.0)>
<Transport(ID='TOUR01-0003', Department='TEST1', Shipments=['CTHO1-46684', 'CTHO1-46513', 'NAESJ-DR307', 'NAESJ-DR112'], Weight=9868.0, Volume=25.520000000000003, Ldm=12.8, status=Planning, vehicle=, Haulier=, Driver=, Trailer=, Cost=11500.0)>


In [ ]:
Transport_assign(Truck.get_by_id("EO92291"), Transport.get_by_id("TOUR01-0001") )

Transport TOUR01-0001 assigned to Truck EO92291.
Updated Transport details: Vehicle=EO92291, Driver=Anders Dam, Trailer=BN2545, Haulier=Dilligencen ApS
Updated Truck details: Transport=TOUR01-0001, Trailer=BN2545


(<Transport(ID='TOUR01-0001', Department='TEST1', Shipments=['BOLIA-D4732', 'BOLIA-D4497'], Weight=5638.0, Volume=76.71100000000001, Ldm=19.2, status=Planning, vehicle=EO92291, Haulier=Dilligencen ApS, Driver=Anders Dam, Trailer=BN2545, Cost=15700.0)>,
 <Truck(License_plate='EO92291', Driver='Anders Dam', Haulier='Dilligencen ApS', Transport='TOUR01-0001', Trailer='BN2545', Location='DK-6100')>)

In [ ]:
Transport_unassign(Truck.get_by_id("EO92291"),Transport.get_by_id("TOUR01-0001") )

Transport TOUR01-0001 successfully unassigned from Truck EO92291.
Updated Transport details: Vehicle='', Driver='', Haulier='', Trailer=''
Updated Truck details: Transport='', Trailer='BN2545' (unchanged from truck's original trailer)


(<Transport(ID='TOUR01-0001', Department='TEST1', Shipments=['BOLIA-D4732', 'BOLIA-D4497'], Weight=5638.0, Volume=76.71100000000001, Ldm=19.2, status=Planning, vehicle=, Haulier=, Driver=, Trailer=, Cost=15700.0)>,
 <Truck(License_plate='EO92291', Driver='Anders Dam', Haulier='Dilligencen ApS', Transport='', Trailer='BN2545', Location='DK-6100')>)

In [ ]:
Transport.get_by_id("TOUR01-0003").Stops

[<Stop(ID='CTHO1-46684_P', Type='P', City='ODENSE S', Date='2026-01-19')>,
 <Stop(ID='CTHO1-46513_P', Type='P', City='ODENSE S', Date='2026-01-19')>,
 <Stop(ID='CTHO1-46684_D', Type='D', City='VINDERUP', Date='2026-01-20')>,
 <Stop(ID='CTHO1-46513_D', Type='D', City='ROSKILDE', Date='2026-01-20')>,
 <Stop(ID='NAESJ-DR307_P', Type='P', City='SKJERN', Date='2026-01-16')>,
 <Stop(ID='NAESJ-DR112_P', Type='P', City='SKJERN', Date='2026-01-16')>,
 <Stop(ID='NAESJ-DR307_D', Type='D', City='KØGE', Date='2026-01-19')>,
 <Stop(ID='NAESJ-DR112_D', Type='D', City='KØGE', Date='2026-01-19')>]

In [ ]:
List_of_Shipments.to_csv('/content/drive/MyDrive/Driver app/List_of_Shipments.csv', index=False)
df_trucks.to_csv('/content/drive/MyDrive/Driver app/df_trucks.csv', index=False)
print("List_of_Shipments saved to /content/drive/MyDrive/Driver app/List_of_Shipments.csv")
print("df_trucks saved to /content/drive/MyDrive/Driver app/df_trucks.csv")

List_of_Shipments saved to /content/drive/MyDrive/Driver app/List_of_Shipments.csv
df_trucks saved to /content/drive/MyDrive/Driver app/df_trucks.csv


In [ ]:
df_from_db.to_csv('/content/drive/MyDrive/Driver app/all_shipments.csv', index=False)
print("All shipments saved to /content/drive/MyDrive/Driver app/all_shipments.csv")

All shipments saved to /content/drive/MyDrive/Driver app/all_shipments.csv


In [ ]:
import pandas as pd

# Convert Trailer.registry to a list of dictionaries for DataFrame creation
data_for_trailer_df = []
for license_plate, trailer_obj in Trailer.registry.items():
    data_for_trailer_df.append({
        'License_plate': trailer_obj.License_plate,
        'Type': trailer_obj.Type,
        'Sub_type': trailer_obj.Sub_type,
        'Department': trailer_obj.Department,
        'Open_pool': trailer_obj.Open_pool,
        'Transport': trailer_obj.Transport,
        'Start_date': trailer_obj.Start_date,
        'End_date': trailer_obj.End_date
    })

# Create the DataFrame
df_trailers = pd.DataFrame(data_for_trailer_df)

# Save the DataFrame to a CSV file on Google Drive
df_trailers.to_csv('/content/drive/MyDrive/Driver app/df_trailers.csv', index=False)

print(f"Successfully saved {len(df_trailers)} trailers to /content/drive/MyDrive/Driver app/df_trailers.csv")
display(df_trailers.head())

Successfully saved 50 trailers to /content/drive/MyDrive/Driver app/df_trailers.csv


,License_plate,Type,Sub_type,Department,Open_pool,Transport,Start_date,End_date
0,WC8195,Curtain,,TEST1,False,None,None,None
1,RB6987,Box,,TEST1,False,None,None,None
2,LF4648,Curtain,Raisable,TEST1,False,None,None,None
3,PP1053,Box,Raisable,TEST1,False,None,None,None
4,LF0217,Reefer,Raisable,TEST1,False,None,None,None
